In [1]:
%pip install ultralytics
%pip install super-gradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 10.0 MB/

In [2]:
from super_gradients.training import Trainer, dataloaders, models
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train,
    coco_detection_yolo_format_val
)

from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import (
    DetectionMetrics_050,
    DetectionMetrics_050_095
)
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
import os
import requests
import zipfile
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import random
import gc

[2023-11-02 14:28:23] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2023-11-02 14:28:23] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-11-02 14:28:23] INFO - utils.py - NumExpr defaulting to 2 threads.
[2023-11-02 14:28:37] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-11-02 14:28:37] WARNING - export.py - Failed to import pytorch_quantization
[2023-11-02 14:28:37] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-11-02 14:28:37] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2023-11-02 14:28:37] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2023-11-02 14:28:37] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2023-11-02 14:28:37] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not found
[2023-11-02 14:28:37] WARNING - env_sanity_check.py - Failed to verify i

In [2]:
from ultralytics import YOLO
from google.colab import files
from google.colab import drive
import locale
drive.mount('/content/drive', force_remount=True)
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

ModuleNotFoundError: ignored

In [ ]:
folder_path = '/content/drive/MyDrive/dataset/labels/val'
output_file_path = '/content/drive/MyDrive/modified_dataset/labels/val'
output_file_path = '/content/drive/MyDrive/dataset/labels/val'
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        #output_file_path=os.path.join(folder_path, file_name)
        with open(file_path, 'r+') as file, open(file_path, 'w') as output_file:
          for line in file:
            if len(line) >= 3:
              modified_line = line[:1] + line[3:]
              output_file.write(modified_line)
              print(f"Modified {file_name}")
            else:
              output_file.write(line)
              print(f"File {file_name} is too short and was not modified.")


In [ ]:
folder_path = '/content/drive/MyDrive/dataset/labels/train'
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        #output_file_path=os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
          lines = file.readlines()

        modified_lines = []
        for line in lines:
          if len(line) >= 3:
            modified_line = line[:1] + line[3:]
            modified_lines.append(modified_line)
          else:
            modified_lines.append(line)
        with open(file_path, 'w') as file:
            file.writelines(modified_lines)
            print(f"Characters removed from {file_path}")

Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars133.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars108.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars17.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars126.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars149.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars165.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars159.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars157.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars180.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars186.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars107.txt
Characters removed from /content/drive/MyDrive/dataset/labels/train/Cars154.txt
Characters removed from /content/drive/My

In [4]:
DATA_DIR='/content/drive/MyDrive/dataset'
train_imgs_dir='/content/drive/MyDrive/dataset/images/train'
train_lbls_dir='/content/drive/MyDrive/dataset/labels/train'
val_imgs_dir='/content/drive/MyDrive/dataset/images/val'
val_lbls_dir='/content/drive/MyDrive/dataset/labels/val'
test_imgs_dir='/content/drive/MyDrive/dataset/images/test'
test_lbls_dir='/content/drive/MyDrive/dataset/labels/test'
classes=['license']

In [5]:
dataset_params={
    'data_dir':DATA_DIR,
    'train_images_dir':train_imgs_dir,
    'train_labels_dir':train_lbls_dir,
    'val_images_dir':val_imgs_dir,
    'val_labels_dir':val_lbls_dir,
    'test_images_dir':test_imgs_dir,
    'test_labels_dir':test_lbls_dir,
    'classes':classes
}

In [6]:
#glob params
epochs=30
batch_size=16
workers=8

In [7]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir':dataset_params['data_dir'],
        'images_dir':dataset_params['train_images_dir'],
        'labels_dir':dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':batch_size,
        'num_workers':workers
    }
)
val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir':dataset_params['data_dir'],
        'images_dir':dataset_params['val_images_dir'],
        'labels_dir':dataset_params['val_labels_dir'],
        'classes':dataset_params['classes']
    },
    dataloader_params={
        'batch_size':batch_size,
        'num_workers':workers
    }
)

Mounted at /content/drive
UTF-8


[2023-11-02 14:31:09] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 389/389 [01:50<00:00,  3.53it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
[2023-11-02 14:32:59] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:  98%|█████████▊| 43/44 [00:22<00:00,  1.95it/s]

In [8]:
train_params = {
    'silent_mode':False,
    'average_best_models':True,
    'warmup_mode':'linear_epoch_step',
    'warmup_initial_lr':1e-6,
    'lr_warmup_epochs':3,
    'initial_lr':5e-4,
    'lr_mode':'cosine',
    'cosine_final_lr_ratio':0.1,
    "train_metrics_list": ["Accuracy", "Top5"],
    "val_metrics_list": ["Accuracy", "Top5"],
    "metric_to_watch": "Accuracy",
    'optimizer':'Adam',
    'optimizer_params':{'weight_decay':0.0001},
    'zero_weight_decay_on_bias_and_bn':True,
    'ema':True,
    'ema_params':{'decay':0.9, 'decay_type':'threshold'},
    'max_epochs': epochs,
    'mixed_precision':True,
    'loss':PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    'valid_metrics_list': [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        ),
        DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    'metrics_to_watch':'mAP@0.50:0.95'
}

In [9]:
models_to_train=[
    'yolo_nas_s'
]
    #'yolo_nas_m',
    #'yolo_nas_l'

In [10]:
checkpoint_root_dir='checkpoint'

In [11]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": epochs,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [12]:
for model_to_train in models_to_train:
  trainer=Trainer(
      experiment_name=model_to_train,
      ckpt_root_dir=checkpoint_root_dir
  )
  model=models.get(
      model_to_train,
      num_classes=len(dataset_params['classes']),
      pretrained_weights='coco'
  )
  trainer.train(
      model=model,
      training_params=train_params,
      train_loader=train_data,
      valid_loader=val_data
  )

Indexing dataset annotations: 100%|██████████| 44/44 [00:22<00:00,  1.94it/s]
[2023-11-02 14:33:23] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth
100%|██████████| 73.1M/73.1M [00:00<00:00, 215MB/s]
[2023-11-02 14:33:24] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s
[2023-11-02 14:33:31] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20231102_143331_632623`
[2023-11-02 14:33:31] INFO - sg_trainer.py - Checkpoints directory: checkpoint/yolo_nas_s/RUN_20231102_143331_632623
[2023-11-02 14:33:31] INFO - sg_trainer.py - Using EMA with params {'decay': 0

The console stream is now moved to checkpoint/yolo_nas_s/RUN_20231102_143331_632623/console_Nov02_14_33_31.txt


/usr/local/lib/python3.10/dist-packages/super_gradients/common/registry/registry.py:72: DeprecationWarning: Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
  warnings.warn(f"Object name `{name}` is now deprecated. Please replace it with `{deprecated_names[name]}`.", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
[2023-11-02 14:33:58] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the mac

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.8056
│   ├── Ppyoloeloss/loss_iou = 0.3179
│   ├── Ppyoloeloss/loss_dfl = 1.5409
│   └── Ppyoloeloss/loss = 3.3709
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.9887
    ├── Ppyoloeloss/loss_iou = 0.2663
    ├── Ppyoloeloss/loss_dfl = 1.4095
    ├── Ppyoloeloss/loss = 3.3591
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0012
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 3/3 [00:03<00:00,  1.22s/it]
[2023-11-02 14:35:53] INFO - base_sg_logger.py - Checkpoint saved in checkpoint/yolo_nas_s/RUN_20231102_143331_632623/ckpt_best.pth
[2023-11-02 14:35:53] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.500390350818634


...

Validating epoch 28: 100%|██████████| 3/3 [00:04<00:00,  1.51s/it]


SUMMARY OF EPOCH 28
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7857
│   │   ├── Epoch N-1      = 0.7988 (↘ -0.0131)
│   │   └── Best until now = 0.7958 (↘ -0.0102)
│   ├── Ppyoloeloss/loss_iou = 0.1779
│   │   ├── Epoch N-1      = 0.185  (↘ -0.0071)
│   │   └── Best until now = 0.1761 (↗ 0.0018)
│   ├── Ppyoloeloss/loss_dfl = 0.8859
│   │   ├── Epoch N-1      = 0.8997 (↘ -0.0138)
│   │   └── Best until now = 0.8853 (↗ 0.0006)
│   └── Ppyoloeloss/loss = 1.6734
│       ├── Epoch N-1      = 1.7112 (↘ -0.0378)
│       └── Best until now = 1.6832 (↘ -0.0098)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.7853
    │   ├── Epoch N-1      = 0.7618 (↗ 0.0234)
    │   └── Best until now = 0.7557 (↗ 0.0295)
    ├── Ppyoloeloss/loss_iou = 0.2021
    │   ├── Epoch N-1      = 0.2094 (↘ -0.0073)
    │   └── Best until now = 0.2014 (↗ 0.0006)
    ├── Ppyoloeloss/loss_dfl = 0.9875
    │   ├── Epoch N-1      = 1.0007 (↘ -0.0132)
    │   └── Best until now = 0.9891 (↘ -0.0016)
    ├── Ppyoloeloss/loss =

Validating epoch 29: 100%|██████████| 3/3 [00:04<00:00,  1.54s/it]


SUMMARY OF EPOCH 29
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7835
│   │   ├── Epoch N-1      = 0.7857 (↘ -0.0022)
│   │   └── Best until now = 0.7857 (↘ -0.0022)
│   ├── Ppyoloeloss/loss_iou = 0.1736
│   │   ├── Epoch N-1      = 0.1779 (↘ -0.0044)
│   │   └── Best until now = 0.1761 (↘ -0.0026)
│   ├── Ppyoloeloss/loss_dfl = 0.8659
│   │   ├── Epoch N-1      = 0.8859 (↘ -0.02)
│   │   └── Best until now = 0.8853 (↘ -0.0194)
│   └── Ppyoloeloss/loss = 1.6504
│       ├── Epoch N-1      = 1.6734 (↘ -0.0231)
│       └── Best until now = 1.6734 (↘ -0.0231)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.7991
    │   ├── Epoch N-1      = 0.7853 (↗ 0.0138)
    │   └── Best until now = 0.7557 (↗ 0.0433)
    ├── Ppyoloeloss/loss_iou = 0.2037
    │   ├── Epoch N-1      = 0.2021 (↗ 0.0017)
    │   └── Best until now = 0.2014 (↗ 0.0023)
    ├── Ppyoloeloss/loss_dfl = 0.9793
    │   ├── Epoch N-1      = 0.9875 (↘ -0.0082)
    │   └── Best until now = 0.9875 (↘ -0.0082)
    ├── Ppyoloeloss/loss = 

[2023-11-02 15:09:47] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 30: 100%|██████████| 3/3 [00:03<00:00,  1.09s/it]


In [13]:
gc.collect()

1207

In [18]:
model.export('yolo-nas_export',format='onnx')

TypeError: ignored

In [1]:
!zip -r /content/yolo-nas_30e.zip /content/checkpoint/yolo_nas_s
files.download("/content/yolo-nas_30e.zip")

	zip warning: name not matched: /content/checkpoint/yolo_nas_s

zip error: Nothing to do! (try: zip -r /content/yolo-nas_30e.zip . -i /content/checkpoint/yolo_nas_s)


NameError: ignored